In [26]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from generator import DataGenerator
PROJECT_NAME = "LPRnet_keras"
MODEL_NAME = "depthwise_model_randomchars_perspective_tflite"

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1

In [11]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss

In [12]:
class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.SeparableConv2D(out_div4,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(3,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(1,3),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_channels,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]  
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

In [13]:
#test this later

class global_context(keras.layers.Layer):
    def __init__(self,kernel_size,stride,**kwargs):
        super().__init__(**kwargs)
        self.ksize = kernel_size
        self.stride = stride

    def call(self, input):
        x = input 
        avg_pool = keras.layers.AveragePooling2D(pool_size=self.ksize,strides=self.stride,padding='same')(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(avg_pool)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        out = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([avg_pool , sqm])
        #out = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(avg_pool)
        return out

    def get_config(self):
        return {
            "kernel_size": self.ksize,
            "stride": self.stride,
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [14]:
class LPRnet(keras.Model):
    def __init__(self, input_shape=(24,94,3), **kwargs):
        super(LPRnet, self).__init__(**kwargs)
        self.input_layer = keras.layers.Input(input_shape)
        self.cnn_layers= [
            keras.layers.SeparableConv2D(64,kernel_size = (3,3),strides=1,padding='same',name='main_conv1',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(name='BN1'),
            keras.layers.ReLU(name='RELU1'),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,1),name='maxpool2d_1',padding='same'),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_2',padding='same'),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_3',padding='same'),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(256,(4,1),strides=1,padding='same',name='main_conv2',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(NUM_CLASS,(1,13),padding='same',name='main_conv3',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]
        self.out_layers = [
            keras.layers.SeparableConv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
        ]
        self.out = self.call(self.input_layer)

    def call(self,inputs,training=False):
        x = inputs
        layer_outputs = []
        for layer in self.cnn_layers:
            x = layer(x)
            layer_outputs.append(x)
        scale1 = global_context((1,4),(1,4))(layer_outputs[0])
        scale2 = global_context((1,4),(1,4))(layer_outputs[4])
        scale3 = global_context((1,2),(1,2))(layer_outputs[6])
        scale5 = global_context((1,2),(1,2))(layer_outputs[7])
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(x)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        scale4 = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([x , sqm])
        gc_concat = keras.layers.Lambda(lambda x: tf.concat([x[0], x[1], x[2], x[3], x[4]],3))([scale1, scale2, scale3, scale5,scale4])
        for layer in self.out_layers:
            gc_concat = layer(gc_concat)
        logits = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x[0],axis=1))([gc_concat])
        logits = keras.layers.Softmax()(logits)
        return logits

In [15]:
import glob
import cv2

real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')

data = []
labels = []
val_data = []
val_labels = []

for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))/256
    data.append(image)
    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

for file in real_images_val:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

training_set = np.array(data,dtype=np.float32)
training_labels = np.array(labels)
ragged = tf.ragged.constant(training_labels).to_tensor()
real_dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).batch(64)


C:\Users\carlos\AppData\Local\Temp/ipykernel_676/2036217571.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(labels)


In [28]:
if os.path.exists("trained_models//{}".format(MODEL_NAME)):
    model = keras.models.load_model(
        "trained_models//{}".format(MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
    )
    print("loading model")
else:
    model = LPRnet()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
    model.build((1,24,94,3))
    print("creating model")

loading model


In [22]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset
test_dataset = generate_dataset()

C:\Users\carlos\AppData\Local\Temp/ipykernel_676/2355684483.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


In [23]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project=MODEL_NAME, entity="clsandoval")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 400,
  "batch_size": 64
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: clsandoval (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [24]:
generate = DataGenerator()
check = tf.keras.callbacks.ModelCheckpoint(
    './trained_models/{}'.format(MODEL_NAME),
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq=500,
    options=None,
)
model.fit(test_dataset)
model.fit_generator(generator=generate,validation_data=real_dataset,validation_steps=3,epochs=11,steps_per_epoch=50,callbacks=[check])


2/2 [==============================] - 5s 1s/step - loss: 61.8105
Epoch 1/11


C:\Users\carlos\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
c:\Users\carlos\Desktop\cs\ml-sandbox\ANPR\LPRnet-keras\generator.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


50/50 [==============================] - 14s 253ms/step - loss: 28.8266 - val_loss: 100.2028
Epoch 2/11
50/50 [==============================] - 11s 209ms/step - loss: 22.9276 - val_loss: 100.2028
Epoch 3/11
50/50 [==============================] - 11s 214ms/step - loss: 22.5947 - val_loss: 100.2028
Epoch 4/11
50/50 [==============================] - ETA: 0s - loss: 22.3106

KeyboardInterrupt: 

In [8]:

TFLITE_PATH = 'tflite_models'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("./{}/{}.tflite".format(TFLITE_PATH,MODEL_NAME), 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpekd_340s\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmpekd_340s\assets


In [18]:
interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format('recognition'))
#interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\keras_lprnet_separable.tflite")
import numpy as np
import time 
import glob
import cv2

interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
start = time.time()
ctr = 0
batch_images=[]
batch_labels = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    batch_labels.append(label)
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = test_image.astype(np.float32)
    batch_images.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,),greedy=False)
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        print(text, " "+ label)
        ctr += 1
print(time.time()-start)
print(ctr,len(real_images))

DAJ1083  DAJ1083
DCP7079  DCP7079
NCH8029  NCH8029
NCM9415  NCM9415
NFX7563  NFX7563
NGS2592  NGS2592
DAG3746  DAG3746
DAH3150  DAH3150
DAK1304  DAK1304
DAL7407  DAL7407
DCP5431  DCP5431
DCP7323  DCP7323
NA36503  NA36503
NBK6857  NBK6857
NBR9407  NBR9407
NC04583  NC04583
NCM8042  NCM8042
NCM9147  NCM9147
NDE9713  NDE9713
NDF2712  NDF2712
NDS1024  NDS1024
NED3275  NED3275
CAB1069  CAB1069
NA02641  NA02641
NBC2309  NBC2309
NDT7127  NDT7127
NBT2564  NBT2564
NDT8216  NDT8216
CDZ5495  CDZ5495
NAT6342  NAT6342
NDC2372  NDC2372
AXA3085  AXA3085
NBE1565  NBE1565
NGS8307  NGS8307
DAT1060  DAT1060
GCS7606  GCS7606
NAR3508  NAR3508
NAT1827  NAT1827
NBA9743  NBA9743
NCR8656  NCR8656
NFY2439  NFY2439
AKA9280  AKA9280
DAC3413  DAC3413
NBV6196  NBV6196
NCW7897  NCW7897
DCP1485  DCP1485
NGY2496  NGY2496
DAC7436  DAC7436
DAE1045  DAE1045
NAE2616  NAE2616
NDX8504  NDX8504
DAB1391  DAB1391
DAL8038  DAL8038
DAU1856  DAU1856
NCA1238  NCA1238
NCB7161  NCB7161
NEA9103  NEA9103
NED8751  NED8751
NEF1096  NEF10

In [4]:
tflite_interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\{}.tflite".format(MODEL_NAME))
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (193, 24, 94, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (193, 24,36))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])
start = time.time()
batch = np.array(batch_images)
tflite_interpreter.set_tensor(input_details[0]['index'], batch)
tflite_interpreter.invoke()
output_data = tflite_interpreter.get_tensor(output_details[0]['index'])
ctr = 0
for k,v in enumerate(output_data):
        text = ""
        print(v.shape)
        undecoded=np.expand_dims(v,axis=0)
        decoded = keras.backend.ctc_decode(undecoded,(24,),greedy=True)
        for j in np.array(decoded[0]).reshape(24):
            if j >-1:
                text += DECODE_DICT[j]
        if batch_labels[k] == text:
            print(text)
            ctr += 1
print(ctr)
print(time.time()-start)
print(ctr,len(real_images))

== Input details ==
name: inputs
shape: [193  24  94   3]
type: <class 'numpy.float32'>

== Output details ==
name: Identity
shape: [193  24  36]
type: <class 'numpy.float32'>
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
(24, 36)
DAF9190
(24, 36)
DAJ1083
(24, 36)
(24, 36)
DCP7079
(24, 36)
(24, 36)
NCG1221
(24, 36)
NCH8029
(24, 36)
(24, 36)
(24, 36)
NFX7563
(24, 36)
(24, 36)
(24, 36)
(24, 36)
AAW1583
(24, 36)
(24, 36)
(24, 36)
(24, 36)
ARA6828
(24, 36)
(24, 36)
(24, 36)
(24, 36)
DAG3746
(24, 36)
DAH3150
(24, 36)
(24, 36)
DAL7407
(24, 36)
(24, 36)
(24, 36)
DCP5431
(24, 36)
DCP7323
(24, 36)
NA36503
(24, 36)
(24, 36)
NAE2008
(24, 36)
(24, 36)
NBA4750
(24, 36)
NBK6857
(24, 36)
(24, 36)
NBR9407
(24, 36)
NC04583
(24, 36)
(24, 36)
NCM8042
(24, 36)
NCM9147
(24, 36)
NDE9713
(24, 36)
(24, 36)
NDJ3111
(24, 36)
NDS1024
(24, 36)
(24, 36)
NEA4294
(24, 36)
(24, 36)
NED3275
(24, 36)
(24, 36)
NGJ4631
(24, 36)
(24, 36)
(24, 36)
(24, 36)
CAB1069
(24, 36)
NA02641
(24, 36)
NBC2309
(24, 36)
(24, 36)
NDT7127


In [6]:
MODEL_PATH = 'trained_models'
TFLITE_PATH = 'tflite_models'
model = keras.models.load_model(
    os.path.join(MODEL_PATH, MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
)

Model: "lp_rnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_conv1 (SeparableConv2D) multiple                  283       
_________________________________________________________________
BN1 (BatchNormalization)     multiple                  256       
_________________________________________________________________
RELU1 (ReLU)                 multiple                  0         
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   multiple                  0         
_________________________________________________________________
small_basic_block (small_bas multiple                  9600      
_________________________________________________________________
maxpool2d_2 (MaxPooling2D)   multiple                  0         
_________________________________________________________________
small_basic_block_1 (small_b multiple                  3558

In [29]:
import glob
import cv2
real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\*\\*\\*.png')
ctr = 0
batch = []
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    image = cv2.imread(file).astype('float32')
    test_image = cv2.resize(image,(94,24))/256
    batch.append(test_image)
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text or text[1:] == label or text[:-1] == label:
        ctr += 1
    print(text," "+ label)
print(ctr)
print(len(real_images))

587S05LAA  07303
7301Q85B125  1303
130340843  1303
566TPEAA  566TPE
35TNMSA  57
EE800NGMAA  59
A103573AA  AA03573
AAK17053A  AAK7053
AAE4G73A  AAL4673
ATG3TN5A  ACG3105
AB352S3A  AD35293
DAL5017A  DAL5017
DAMA00  DAM7900
DC18721  DC18721
G1GT5EA  G2G725
WAF6171  NAF6171
NBK7446  NBK7446
MG5725ZA  NC57254
NCG109Z  NCG1092
NCZ5306  NCZ5306
SS83253GA  NDB3253
NE70G3GA  NE70636
NE70G3GA  NE84727
NEJ9900  NEJ9900
MF2Y85A  NH
PYY50TA  PYV501
WSNMBAA  TJX496
VW50T54A  WSO154
ZUVZ5TAA  ZJV257
TG30QRDR955A  1303
943N0XA  943NOX
UDAE3011  DAE3011
NAH7401  NAH7401
W8G1384  NBG1384
NBV2323U  NBV2323
NC16514  NCI6514
FNCJ9396  NCJ9396
LNCU375B  NCU3758
ND11397  NDI1397
4SQT580E7AA  NDT5180
NDU6721A  NDU6721
ZNET1791A  NET1793
NGG6277A  NGG6277
TTXSKQTAA  TYX907
0GEB09AA  UGF809
BET339AA  WET339
659RZUFA  XSC203
Y5G054  YS6054
AAH7062  AAH7062
AAZ2G55  AAZ2655
ABG5953  ABG5953
ACZEQBAA  ACZ4349
2P1BT75A  APA8716
NAD1140  NAD1140
N86M27805  NBQ2703
TNCY8S40  NCY8340
NDG3354  NDG3354
UU2B45A  UIT846
0

In [24]:
val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).batch(64)

C:\Users\carlos\AppData\Local\Temp/ipykernel_21744/3660101917.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_training_labels = np.array(val_labels)


In [12]:
preds = model.predict(batch)


In [32]:
ctr = 0
for c,file in zip(preds, real_images):
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    z = np.expand_dims(c,axis=0)
    decoded = tf.keras.backend.ctc_decode(z,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text or text[1:] == label or text[:-1] == label:
        ctr += 1
    print(text," "+ label)
print(ctr)
print(len(real_images))

MEETESYAA  07303
Y5DV5H426  1303
0
1
130320843  1303
2
3
566TPEAA  566TPE
3SU79MQA  57
EPBMUMMAA  59
A103513AA  AA03573
AK7053  AAK7053
AAL4613A  AAL4673
WFN3YH5  ACG3105
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
AD35293A  AD35293
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
DAL5017A  DAL5017
DAMY900  DAM7900
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
DC18721A  DC18721
G2GT55AAA  G2G725
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
NAF6171  NAF6171
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
NBK7446  NBK7446
NG57352A  NC57254
NCG102  NCG1092
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
NCZ5306  NCZ5306
N0U32S3A  NDB3253
112
113
NE70636AA  NE70636
NE70636AA  NE84727
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
NEJ9900A  NEJ9900
NPJ2785YAA  NH
PYV51AA  PYV501
WJNRK9PHA  TJX496
WS01154A  WSO154
132
133
ZJV257AA  ZJV257
QGH9A  1303
943N0XAA  943NOX
134
135
136
137
138
139
140
141
142
143
144
145
146

In [34]:
import glob
images =  glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\*\\*\\*.png')
print(len(images))

def data_gen():
    a = []
    for i in range(500):
        filename = images[i]
        img = cv2.imread(filename)
        img = cv2.resize(img, (94,24))
        img = img / 255.0
        img = img.astype('float32')
        a.append(img)
    a = np.array(a)
    print(a.shape) # a is np array of 160 3D images
    img = tf.data.Dataset.from_tensor_slices(a).batch(1)
    for i in img.take(32):
        print(i)
        yield [i]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
if True:
    # converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # converter.representative_dataset = representative_data_gen
    # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # converter.inference_input_type = tf.uint8
    # converter.inference_output_type = tf.uint8
    converter.representative_dataset = data_gen
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.SELECT_TF_OPS]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    converter.allow_custom_ops = True

    print(converter.experimental_new_quantizer)  # outputs True
    print(converter.experimental_new_converter)  # outputs True
else:
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter.optimizations = []

tflite_model = converter.convert()

with open('quant.tflite', 'wb') as f:
    f.write(tflite_model)

786
True
True


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp94kgnn4s\assets


INFO:tensorflow:Assets written to: C:\Users\carlos\AppData\Local\Temp\tmp94kgnn4s\assets


(500, 24, 94, 3)
tf.Tensor(
[[[[0.1254902  0.14117648 0.1764706 ]
   [0.1254902  0.13725491 0.18039216]
   [0.11764706 0.14901961 0.22352941]
   ...
   [0.14117648 0.6901961  0.8352941 ]
   [0.         0.29411766 0.40784314]
   [0.04313726 0.12941177 0.21568628]]

  [[0.11372549 0.14117648 0.18431373]
   [0.11764706 0.14509805 0.1882353 ]
   [0.11764706 0.14901961 0.21176471]
   ...
   [0.20392157 0.59607846 0.7294118 ]
   [0.05490196 0.3019608  0.41568628]
   [0.18431373 0.27058825 0.35686275]]

  [[0.23921569 0.26666668 0.31764707]
   [0.23921569 0.26666668 0.31764707]
   [0.22745098 0.2627451  0.3254902 ]
   ...
   [0.14117648 0.32156864 0.44705883]
   [0.1254902  0.29411766 0.39607844]
   [0.18039216 0.29803923 0.39215687]]

  ...

  [[0.6039216  0.6666667  0.77254903]
   [0.627451   0.73333335 0.89411765]
   [0.4117647  0.6509804  0.8509804 ]
   ...
   [0.5686275  0.627451   0.9529412 ]
   [0.6627451  0.68235296 0.9529412 ]
   [0.6313726  0.5568628  0.73333335]]

  [[0.6392157  0.

tf.Tensor(
[[[[0.21176471 0.22745098 0.23529412]
   [0.21176471 0.22745098 0.23529412]
   [0.21176471 0.22745098 0.23529412]
   ...
   [0.04313726 0.04313726 0.04313726]
   [0.03529412 0.03529412 0.03529412]
   [0.03137255 0.03137255 0.03137255]]

  [[0.21568628 0.23137255 0.23921569]
   [0.21176471 0.22745098 0.23529412]
   [0.21176471 0.22745098 0.23529412]
   ...
   [0.03921569 0.03921569 0.03921569]
   [0.03137255 0.03137255 0.03137255]
   [0.02352941 0.02352941 0.02352941]]

  [[0.22352941 0.23137255 0.23921569]
   [0.21960784 0.22745098 0.23529412]
   [0.21960784 0.22745098 0.23529412]
   ...
   [0.04313726 0.04313726 0.04313726]
   [0.03137255 0.03137255 0.03137255]
   [0.01960784 0.01960784 0.01960784]]

  ...

  [[0.38039216 0.3647059  0.3764706 ]
   [0.39215687 0.3764706  0.3882353 ]
   [0.40392157 0.3882353  0.4       ]
   ...
   [0.01176471 0.01176471 0.01176471]
   [0.01176471 0.01176471 0.01176471]
   [0.01176471 0.01176471 0.01176471]]

  [[0.23921569 0.23529412 0.243137